In [1]:
import nltk
from nltk.tokenize import RegexpTokenizer, wordpunct_tokenize
from nltk.tokenize import sent_tokenize
import operator
import os
import sklearn
from sklearn.model_selection import train_test_split
import numpy as np
import statistics 
from statistics import mean
from functools import reduce
from decimal import Decimal
import pandas as pd

In [2]:
files= os.listdir('train') #get all file names under the training dataset
all_text = []
all_text2 = ''
for file in files: 
    f = open('train/'+file, mode = 'r')
    article = f.read()
    all_text.append(article.lower()) # lowercase all the words
    all_text2 = all_text2 + article.lower()
    f.close()

In [3]:
def token(text):
    '''
    pattern = r ''' '''(?x)    
         (?:[a-z]\.)+
         | \$?\d+(?:\.\d+)?%?
         | \'?\w+\'?
         | \<\S+\>
         | \w+(?:-\w+)*
         ''' '''
    | [][.,;"'?()~!@#&*+={}/<>:-_`]
    pattern = ('\w+|\$[\d\.]+|\S+|[\d\-]\w+')
    
    tokenizer = RegexpTokenizer('\(?[\w\.]+|\)?')
    '''
    text_token = wordpunct_tokenize(text)
    return text_token

def check_UNK(article, word_dic):
    words_list = token(article)
    for j in words_list:
        if j in word_dic.keys():
            word_dic[j] = word_dic[j]+1 #Count the number that each word appears
        else:
            word_dic[j] = 1 #initiate
    return word_dic

In [4]:
def replace_UNK(word_list, word_dic):
    for n in range(len(word_list)):
        if word_dic[word_list[n]] <= 3:   #Replace wiht UNK if its frequency is too small
            word_list[n] = 'UNK'
    return word_list

In [5]:
def token_sentence(article):
    sentence = sent_tokenize(article)
    return sentence

In [6]:
word_count = {}
for i in all_text:
    word_count = check_UNK(i, word_count) #Update word_count in each iteration

tokened_article_to_sentence = [] # A list contains 2 levels of lists
for i in all_text:
    sentences = token_sentence(i) # token an article to sentences
    tokened_article_to_words = [] # A list contains lists of each sentence from the article
    for j in sentences:
        each_word = token(j) # j is one sentence, token j makes a list of words which form j
        temp = replace_UNK(each_word, word_count) # return a wordlist after replacing by 'UNK'
    
        tokened_article_to_words.append(temp) #add the new wordlist(new sentence with token words) into sentence list 
    tokened_article_to_sentence.append(tokened_article_to_words) #after containing all sentences, append to article list

In [7]:
training_text = []
validation_text = []
for i in tokened_article_to_sentence:
    Sentence_train, Sentence_validation = train_test_split(i, test_size=0.2, random_state=42)
    training_text.extend(Sentence_train)
    validation_text.extend(Sentence_validation)

### 3.1

#### Unigram dictionary and probability dictionary

In [8]:
#Build a dictionary for unigram
def unigram(text_token, unigram_dic):
    for i in text_token:   # i is a list of one sentence token
        if i not in unigram_dic:
            unigram_dic[i] = 1
        else:
            unigram_dic[i] = unigram_dic[i]+1 #Count the number that each word appears
    return unigram_dic


#Build a dictionary for unigram's probability, just use simple count(wi)/N
def unigram_probability(unigram_dic, length):
    unigram_prob_dic = {}
    for j in unigram_dic.keys():
        unigram_prob_dic[j] = unigram_dic[j]/length
    return unigram_prob_dic

#### Bigram dictionary and probability dictionary

In [9]:
#Build a dictionary for bigram
def bigram(text_token, bigram_dic):
    for i in range(len(text_token)):
        if i != 0:
            first_word = text_token[i-1] #Wi-1
            second_word = text_token[i] #Wi
    
            bi_str = second_word + '|' + first_word
            if bi_str not in bigram_dic:
                bigram_dic[bi_str] = 1
            else:
                bigram_dic[bi_str] = bigram_dic[bi_str]+1 #Count the number that each bigram appears(count(Wi-1, Wi))
    return bigram_dic


#Build a dictionary for bigram's probability, use the formulation P(Wi|Wi-1) = count(Wi-1, Wi)/count(Wi-1)
def bigram_probability(unigram_dic, bigram_dic):
    bigram_prob_dic = {}
    for j in bigram_dic.keys():
        bigram_words = j.split('|') #in j, after split, bigram_words[1] is Wi-1, bigram_words[0] is Wi,
        bigram_prob_dic[j] = bigram_dic[j]/unigram_dic[bigram_words[1]]
    return bigram_prob_dic

#### Trigram dictionary and probability dictionary

In [10]:
def trigram(text_token,trigram_dic):
    if len(text_token)>=2:
        for i in range(len(text_token)):
            if i!= 0 and i!=1:
                first_word = text_token[i-2]
                second_word = text_token[i-1]
                third_word = text_token[i]
                tri_str = third_word + '|' +first_word + '||' + second_word
                if tri_str not in trigram_dic:
                    trigram_dic[tri_str] = 1
                else:
                    trigram_dic[tri_str] = trigram_dic[tri_str]+1 #Count the number that each word appears
    return trigram_dic

def trigram_probability(bigram_dic, trigram_dic):
    trigram_prob_dic = {}
    for j in trigram_dic.keys():
        trigram_words = j.split('|',1) #in j, after split, trigram_words[1] is Wi-2||Wi-1, bigram_words[0] is Wi
        trigram_prob_dic[j] = trigram_dic[j]/bigram_dic[trigram_words[1].split('||')[1] + '|' 
                                                    + trigram_words[1].split('||')[0]]
                                        #transfer Wi-2||Wi-1 to Wi|Wi-1 to match the bigram_dic count format
        #Probability of trigram = Count(Wi-2, Wi-1, Wi)/Count(Wi-2, Wi-1)
        #Count(Wi-2, Wi-1) = bigram_dic[trigram_words[1].split('||')[1] + '|' + trigram_words[1].split('||')[0]]
    return trigram_prob_dic

#### Report the unigram, bigram, and trigram wordlevel counts. 

In [12]:
tokened_all_text2 = token(all_text2)
tokened_all_text2 = replace_UNK(tokened_all_text2,word_count)  #Replace 'UNK'
Unigram_counts = dict(sorted(unigram(tokened_all_text2, {}).items(), key=operator.itemgetter(1),reverse=True))
Bigram_counts = dict(sorted(bigram(tokened_all_text2, {}).items(), key=operator.itemgetter(1),reverse=True))
Trigram_counts = dict(sorted(trigram(tokened_all_text2, {}).items(), key=operator.itemgetter(1),reverse=True))


In [13]:
with open('3.1_ngramCounts.txt', 'w') as f:
    f.writelines('A bigram has the form Wi|Wi-1 and a trigram has the form Wi|Wi-2||Wi-1. They were' + 
                 'the count of (Wi-1, Wi) and (Wi-2, Wi-1, Wi). I write in this form for later conveniencely use.\n')
    for i, j in Unigram_counts.items():
        f.writelines(i + ': ' + str(j) + '\n')
    
    for k, l in Bigram_counts.items():
        f.writelines(k + ': ' + str(l) + '\n')
    for m, n in Trigram_counts.items():
        f.writelines(m + ': ' + str(n) + '\n')
    f.close()

### 3.2

#### Build unigram, bigram and trigram dictionary for training dataset, also calculate the probability for each n-gram.

In [14]:
training_unigram_dic = {}# Training data's unigram
training_bigram_dic = {}
training_trigram_dic = {}

training_token_length = 0  # Training data contains how many words


for j in training_text:
    
    training_token_length = training_token_length + len(j)
    #print(temp_token)
    training_unigram_dic = unigram(j, training_unigram_dic)
    training_bigram_dic = bigram(j, training_bigram_dic)
    training_trigram_dic = trigram(j, training_trigram_dic)


training_unigram_dic = dict(sorted(training_unigram_dic.items(), key=operator.itemgetter(1),reverse=True))
training_bigram_dic = dict(sorted(training_bigram_dic.items(), key=operator.itemgetter(1),reverse=True))
training_trigram_dic = dict(sorted(training_trigram_dic.items(), key=operator.itemgetter(1),reverse=True))

In [15]:
training_unigram_prob = unigram_probability(training_unigram_dic, training_token_length)
training_bigram_prob = bigram_probability(training_unigram_dic, training_bigram_dic)
training_trigram_prob = trigram_probability(training_bigram_dic, training_trigram_dic)

#### Validate lambdas

In [18]:
possible_lambdas = []
np.random.seed(255)
for i in range(0,255):
    lam1 = np.random.random_sample()
    lam2 = np.random.random_sample()
    if lam1 != 0 and lam2!=0:
        if lam1 + lam2 < 1:
            lam3 = 1- lam1-lam2
            possible_lambdas.append([lam1,lam2,lam3])


In [22]:
def get_perlexities(text, unigram_prob, bigram_prob, trigram_prob, lambdas):
    #lambda_avgperlexity = {}
    #for lam in lambdas:
    lam1 = lambdas[0]
    lam2 = lambdas[1]
    lam3 = lambdas[2]
    sentence_perlexity = []
    P_sentence = 0
    word_len = 0
    for sentence in text:
        
        for i in range(len(sentence)):
                
            unigram_format = sentence[i]
            if sentence[i] in unigram_prob:
                unigram_format= sentence[i]
                
            else:
                unigram_format = 'UNK'
                
            p_uni = unigram_prob[unigram_format]        
            p_bi = 0
            p_tri = 0
            
            if i == 1:
                if sentence[i-1] in unigram_prob:
                    bi_i_1 = sentence[i-1]
                else:
                    bi_i_1 = 'UNK'
                bigram_format = unigram_format+'|'+ bi_i_1
                if bigram_format in bigram_prob:
                    p_bi = bigram_prob[bigram_format]
                    
            if i != 0 and i !=1:
                if sentence[i-1] in unigram_prob:
                    bi_i_1 = sentence[i-1]
                else:
                    bi_i_1 = 'UNK'
                
                if sentence[i-2] in unigram_prob:
                    tri_i_2 = sentence[i-2]
                else:
                    tri_i_2 = 'UNK'
                
                bigram_format = unigram_format+'|'+ bi_i_1
                trigram_format = unigram_format+'|'+tri_i_2+'||' + bi_i_1
                    
                if bigram_format in bigram_prob:
                    p_bi = bigram_prob[bigram_format]
                    
                if trigram_format in trigram_prob:
                    p_tri = trigram_prob[trigram_format]
                    
            p_word = lam1*p_uni + lam2*p_bi + lam3*p_tri
            
            P_sentence = P_sentence + np.log(p_word)  #p_words_in_a_sentence.append(p_word)
        word_len = word_len + len(sentence)
        
    perplexity = pow((np.e),(-1/word_len)*P_sentence)
    
    return perplexity

In [23]:
lambda_avgperlexity = {}
for i in possible_lambdas:
    lambda_avgperlexity[str(i)] = get_perlexities(validation_text, training_unigram_prob, training_bigram_prob, 
                training_trigram_prob,i)
#dict(sorted(lambda_avgperlexity.items(), key=operator.itemgetter(1),reverse=False))

In [24]:
lambda_avgperlexity = dict(sorted(lambda_avgperlexity.items(), key=operator.itemgetter(1),reverse=False))
lambda_avgperlexity

{'[0.16461176032353309, 0.5598203068649524, 0.2755679328115145]': 113.68533734764159,
 '[0.24804604317255252, 0.4205710076505168, 0.3313829491769307]': 113.75439149777728,
 '[0.20558730582015117, 0.42295571204406324, 0.3714569821357856]': 113.95399371014794,
 '[0.14635694295559765, 0.4984870392538281, 0.35515601779057426]': 114.56578721648206,
 '[0.29980414986898307, 0.4452818326708321, 0.2549140174601848]': 114.62403067274991,
 '[0.1793469896868496, 0.6332920563047046, 0.18736095400844577]': 114.80497852066456,
 '[0.28928328564226646, 0.5089698609823421, 0.20174685337539144]': 114.95417746157044,
 '[0.2500554463581307, 0.3642550576103868, 0.3856894960314825]': 114.99603912629117,
 '[0.1288726154396902, 0.5915665197922657, 0.2795608647680441]': 115.13573231008405,
 '[0.1617098614852347, 0.656512853535566, 0.18177728497919932]': 115.44459839621052,
 '[0.2922873335864258, 0.5254980530126285, 0.1822146134009457]': 115.51204091912614,
 '[0.29810298738832564, 0.526005408378104, 0.1758916042

#### Report each perplexity for each test set

In [25]:
text_files= os.listdir('test') #get all file names under the training dataset
test_articles = []
for file in text_files: 
    f = open('test/'+file, mode = 'r')
    article = f.read()
    test_articles.append(article.lower())
    f.close()

In [26]:
tokened_test_articles = []
for i in test_articles:
    test_sentences = token_sentence(i)
    tokened_test_sentences = []
    for j in test_sentences:
        temp_test_sentence = token(j)
        tokened_test_sentences.append(temp_test_sentence)
    tokened_test_articles.append(tokened_test_sentences)

In [27]:
print('The first test file has perplexity ' + str(get_perlexities(tokened_test_articles[0], training_unigram_prob, training_bigram_prob, 
                training_trigram_prob,[0.16461176032353309, 0.5598203068649524, 0.2755679328115145]))+
      ', while the second test file has perplexity '
      + str(get_perlexities(tokened_test_articles[1], training_unigram_prob, training_bigram_prob, 
                training_trigram_prob,[0.16461176032353309, 0.5598203068649524, 0.2755679328115145])))

The first test file has perplexity 534.8147659788204, while the second test file has perplexity 114.91933361716188


### 3.3 Build another language model with add-λ smoothing. Use λ = 0.1 and λ = 0.3.

In [28]:
def unigram_probability_smoothing(unigram_dic, length, smoothing, V):
    unigram_prob_dic = {}
    for j in unigram_dic.keys():
        unigram_prob_dic[j] = (unigram_dic[j]+smoothing)/(length+V * smoothing)
    return unigram_prob_dic

In [29]:
def bigram_probability_smoothing(unigram_dic, bigram_dic, smoothing, V):
    bigram_prob_dic = {}
    for j in bigram_dic.keys():
        bigram_words = j.split('|') #in j, after split, bigram_words[1] is Wi-1, bigram_words[0] is Wi,
        bigram_prob_dic[j] = (bigram_dic[j]+smoothing)/(unigram_dic[bigram_words[1]]+V*smoothing)
    return bigram_prob_dic

In [30]:
def trigram_probability_smoothing(bigram_dic, trigram_dic, smoothing, V):
    trigram_prob_dic = {}
    for j in trigram_dic.keys():
        trigram_words = j.split('|',1) #in j, after split, trigram_words[1] is Wi-2||Wi-1, bigram_words[0] is Wi
        trigram_prob_dic[j] = (trigram_dic[j]+smoothing)/(bigram_dic[trigram_words[1].split('||')[1] + '|' 
                                                    + trigram_words[1].split('||')[0]] + V*smoothing)
                                        #transfer Wi-2||Wi-1 to Wi|Wi-1 to match the bigram_dic count format
        #Probability of trigram = Count(Wi-2, Wi-1, Wi)/Count(Wi-2, Wi-1)
        #Count(Wi-2, Wi-1) = bigram_dic[trigram_words[1].split('||')[1] + '|' + trigram_words[1].split('||')[0]]
    return trigram_prob_dic

In [31]:
def get_perlexities_smoothing(text, unigram_dic, bigram_dic, trigram_dic, 
                              unigram_prob, bigram_prob, trigram_prob, lambdas, smoothing, V):
    #lambda_avgperlexity = {}
    #for lam in lambdas:
    lam1 = lambdas[0]
    lam2 = lambdas[1]
    lam3 = lambdas[2]
    P_sentence = 0
    word_len = 0
    for sentence in text:
        
        for i in range(len(sentence)):
                
            unigram_format = sentence[i]
            if sentence[i] in unigram_prob:
                unigram_format= sentence[i]
                
            else:
                unigram_format = 'UNK'
                
            p_uni = unigram_prob[unigram_format]
            
            p_bi = smoothing / (smoothing* V)
            p_tri = smoothing / (smoothing* V)
            
            # When calculating the bigram probability with bigram not in training data, consider two conditions: 
            # sentence[i-1] is in the training dataset or not
            if i == 1:
                if sentence[i-1] in unigram_prob:
                    bi_i_1 = sentence[i-1]
                else:
                    bi_i_1 = 'UNK'
                bigram_format = unigram_format+'|'+ bi_i_1
                if bigram_format in bigram_prob:
                    p_bi = bigram_prob[bigram_format]
                else:
                    if bi_i_1 in unigram_dic:
                        p_bi = smoothing / (unigram_dic[bi_i_1] + smoothing* V)
                    
                    
                    
            if i != 0 and i !=1:
                if sentence[i-1] in unigram_prob:
                    bi_i_1 = sentence[i-1]
                else:
                    bi_i_1 = 'UNK'
                
                if sentence[i-2] in unigram_prob:
                    tri_i_2 = sentence[i-2]
                else:
                    tri_i_2 = 'UNK'
                
                bigram_format = unigram_format+'|'+ bi_i_1
                trigram_format = unigram_format+'|'+tri_i_2+'||' + bi_i_1
                    
                if bigram_format in bigram_prob:
                    p_bi = bigram_prob[bigram_format]
                if bigram_format not in bigram_prob:
                    p_bi = smoothing / (unigram_dic[bi_i_1] + smoothing* V)
                    
                    
                # When calculating the trigram probability with trigram not in training data, consider two conditions: 
                # sentence[i-1]|sentence[i-2] is in the training dataset or not    
                if trigram_format in trigram_prob:
                    p_tri = trigram_prob[trigram_format]
                if trigram_format not in trigram_prob:
                    if bi_i_1+"|"+tri_i_2 in bigram_dic:
                        p_tri = smoothing / (bigram_dic[bi_i_1+"|"+tri_i_2] + smoothing* V)
                    
                    
            p_word = lam1*p_uni + lam2*p_bi + lam3*p_tri
            
            P_sentence = P_sentence + np.log(p_word)  #add p_words_in_a_sentence on the existing p_sentence
        word_len = word_len + len(sentence)
        
    perplexity = pow((np.e),(-1/word_len)*P_sentence)
    return perplexity


In [32]:
smoothing_training_unigram_prob_1 = unigram_probability_smoothing(training_unigram_dic, training_token_length, 0.1, len(Unigram_counts))
smoothing_training_bigram_prob_1 = bigram_probability_smoothing(training_unigram_dic, training_bigram_dic,0.1,len(Unigram_counts))
smoothing_training_trigram_prob_1 = trigram_probability_smoothing(training_bigram_dic, training_trigram_dic,0.1,len(Unigram_counts))

In [33]:
print('When λ=0.1, The first test file has perplexity ' + str(get_perlexities_smoothing(tokened_test_articles[0], 
        training_unigram_dic, training_bigram_dic, training_trigram_dic, smoothing_training_unigram_prob_1, smoothing_training_bigram_prob_1, 
                smoothing_training_trigram_prob_1,[0.16461176032353309, 0.5598203068649524, 0.2755679328115145], 0.1, len(Unigram_counts)))
      + ', while the second test file has perplexity '
      + str(get_perlexities_smoothing(tokened_test_articles[1], 
        training_unigram_dic, training_bigram_dic, training_trigram_dic, smoothing_training_unigram_prob_1, smoothing_training_bigram_prob_1, 
                smoothing_training_trigram_prob_1,[0.16461176032353309, 0.5598203068649524, 0.2755679328115145], 0.1, len(Unigram_counts))))

When λ=0.1, The first test file has perplexity 549.0875200787602, while the second test file has perplexity 219.55791746228195


In [34]:
smoothing_training_unigram_prob_3 = unigram_probability_smoothing(training_unigram_dic, training_token_length, 0.3, len(Unigram_counts))
smoothing_training_bigram_prob_3 = bigram_probability_smoothing(training_unigram_dic, training_bigram_dic,0.3,len(Unigram_counts))
smoothing_training_trigram_prob_3 = trigram_probability_smoothing(training_bigram_dic, training_trigram_dic,0.3,len(Unigram_counts))

In [35]:
print('When λ=0.3, The first test file has perplexity ' + str(get_perlexities_smoothing(tokened_test_articles[0], 
        training_unigram_dic, training_bigram_dic, training_trigram_dic, smoothing_training_unigram_prob_3, smoothing_training_bigram_prob_3, 
                smoothing_training_trigram_prob_3,[0.16461176032353309, 0.5598203068649524, 0.2755679328115145], 0.3, len(Unigram_counts)))
      + ', while the second test file has perplexity '
      + str(get_perlexities_smoothing(tokened_test_articles[1], 
        training_unigram_dic, training_bigram_dic, training_trigram_dic, smoothing_training_unigram_prob_3, smoothing_training_bigram_prob_3, 
                smoothing_training_trigram_prob_3,[0.16461176032353309, 0.5598203068649524, 0.2755679328115145], 0.3, len(Unigram_counts))))

When λ=0.3, The first test file has perplexity 629.470804626108, while the second test file has perplexity 295.00018220011725
